<H1>Test Notebook for Stephenson System</H1>

In [1]:
import pyreadr
import pandas as pd
import numpy as np
import datetime
import time
import copy
from stephenson import *

Read R file and convert to pandas df

In [2]:
result = pyreadr.read_r('../data/aflodds.rda')
r_df = list(result.values())[0]
df = pd.DataFrame(r_df)
df.head()

,Date,Week,HomeTeam,AwayTeam,HomeScore,AwayScore,Score,HomeOdds,AwayOdds
0,2009-03-26,1.0,Richmond Tigers,Carlton Blues,67,150,0.0,NaN,NaN
1,2009-03-27,1.0,Hawthorn Hawks,Geelong Cats,103,111,0.0,NaN,NaN
2,2009-03-28,1.0,Collingwood Magpies,Adelaide Crows,86,90,0.0,NaN,NaN
3,2009-03-28,1.0,Brisbane Lions,West Coast Eagles,95,86,1.0,NaN,NaN
4,2009-03-28,1.0,St Kilda Saints,Sydney Swans,80,65,1.0,NaN,NaN


From there select week one outcomes, and create a dataframe of teams and their respective stephenson ratings

In [3]:
week_one = df[(df['Week'] == 1.0)]

combined_teams = pd.concat([week_one['HomeTeam'], week_one['AwayTeam']]) 
all_teams = combined_teams.drop_duplicates().tolist()

team_data = {
    'team': all_teams,   
    'steph_rating': [Stephenson() for _ in all_teams],
}
df = pd.DataFrame(team_data)
df.head()

,team,steph_rating
0,Richmond Tigers,<stephenson.Stephenson object at 0x13b2e07d0>
1,Hawthorn Hawks,<stephenson.Stephenson object at 0x13a6a8c20>
2,Collingwood Magpies,<stephenson.Stephenson object at 0x10ffe3560>
3,Brisbane Lions,<stephenson.Stephenson object at 0x13b2e05f0>
4,St Kilda Saints,<stephenson.Stephenson object at 0x13b2e0620>


Since this is the first outcomes for each team, we don't have a need to update the variance prior to the matchup, however, if this was any other time, uncomment the updateVar with relevant dates. -- Make sure to adjust the rating period to one that suits your system. From there we will adjust the ratings based on the outcome 1 for win 0 for loos .5 for tie. We clone the home rating so when we calculate the away rating we don't use the updated home rating. Then we assign these new ratings to the df

In [4]:
for index, row in week_one.iterrows():
    home_rating = df.loc[df['team'] == row['HomeTeam'], 'steph_rating'].values[0]
    away_rating = df.loc[df['team'] == row['AwayTeam'], 'steph_rating'].values[0]

    # Since this is only week one ignore as its first week
    # home_rating.updateVar(i['Date'])
    # away_rating.updateVar(i['Date'])

    # So we don't evaluate on outcome
    home_clone = copy.deepcopy(home_rating)

    home_rating.newVarRating(away_rating, row['Score'], 1) #pone is 1 for home team - since default gamma is 0 this is irrelevant
    away_rating.newVarRating(home_clone, (1 - row['Score']), 0) 

    df.loc[df['team'] == row['HomeTeam'], 'steph_rating'] = home_rating
    df.loc[df['team'] == row['AwayTeam'], 'steph_rating'] = away_rating

Finally just a test print to show that it returns same results as in R.

In [ ]:
for index, row in df.iterrows():
    print(f"{row['team']}:", end=" ")
    row['steph_rating'].printVals()

Richmond Tigers: Rating: 2065.0272976700026 Deviation: 254.4604126495977
Hawthorn Hawks: Rating: 2065.0272976700026 Deviation: 254.4604126495977
Collingwood Magpies: Rating: 2065.0272976700026 Deviation: 254.4604126495977
Brisbane Lions: Rating: 2334.9727023299974 Deviation: 254.4604126495977
St Kilda Saints: Rating: 2334.9727023299974 Deviation: 254.4604126495977
Melbourne Demons: Rating: 2065.0272976700026 Deviation: 254.4604126495977
Port Adelaide Power: Rating: 2334.9727023299974 Deviation: 254.4604126495977
Fremantle Dockers: Rating: 2065.0272976700026 Deviation: 254.4604126495977
Carlton Blues: Rating: 2334.9727023299974 Deviation: 254.4604126495977
Geelong Cats: Rating: 2334.9727023299974 Deviation: 254.4604126495977
Adelaide Crows: Rating: 2334.9727023299974 Deviation: 254.4604126495977
West Coast Eagles: Rating: 2065.0272976700026 Deviation: 254.4604126495977
Sydney Swans: Rating: 2065.0272976700026 Deviation: 254.4604126495977
North Melbourne Kangaroos: Rating: 2334.972702329